# Experiment comparing simulated data with failure injection vs. GAN-generated failure data

In [1]:
import pandas as pd
import numpy as np
import joblib
import mlflow

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from xgboost import XGBRFClassifier

from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore")

## Preparing the synthetic datasets

In [2]:
df_sim = joblib.load('../outputs/models/train_data.pkl')
df_gan = joblib.load('../inputs/df_gan_generated.pkl')
df_test = joblib.load('../outputs/models/test_data.pkl')

In [3]:
df_sim

,P_Z0_before,P_Z0_after,P_Z1_before,P_Z1_after,P_WH_before,P_WH_after,T_Z0_before,T_Z0_after,T_Z1_before,T_Z1_after,...,P_Z1_delta,P_Z0_delta,P_WH_delta,P_bottom_delta,T_WH_delta,command_type_0,command_type_1,command_type_2,command_type_3,command_type_4
0,782.991774,781.513238,721.997335,732.645404,228.072793,222.739237,88.990443,88.017064,88.256267,91.844784,...,10.648068,-1.478536,-5.333556,-69.947280,1.747713,1.0,0.0,0.0,0.0,0.0
1,823.407968,836.298427,785.214298,786.737540,114.380874,109.468553,85.859813,88.941707,90.356803,87.281548,...,1.523242,12.890459,-4.912321,101.261721,-0.041565,0.0,0.0,0.0,1.0,0.0
2,783.162356,781.606394,727.905276,727.690796,224.554990,230.421390,89.138396,88.773388,89.711909,88.672596,...,-0.214480,-1.555962,5.866400,-0.852047,-1.229892,0.0,0.0,1.0,0.0,0.0
3,782.434641,799.091495,725.812266,725.502688,217.380665,222.701668,88.840552,89.561738,88.047662,87.951192,...,-0.309578,16.656854,5.321003,-75.381007,-3.491265,1.0,0.0,0.0,0.0,0.0
4,845.024456,830.298714,778.727926,791.655508,105.516327,111.314215,85.680819,86.664650,89.330981,86.211135,...,12.927583,-14.725742,5.797888,11.028805,-2.603948,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,787.271113,781.998460,710.195918,716.790706,227.571746,222.070860,88.276114,86.481481,88.353872,88.491776,...,6.594787,-5.272653,-5.500886,-45.014137,-5.753539,0.0,1.0,0.0,0.0,0.0
9996,810.406086,794.551916,752.353625,750.208143,228.402031,224.036259,88.164860,87.298385,88.116074,90.366115,...,-2.145482,-15.854170,-4.365773,37.107424,5.332383,0.0,0.0,0.0,0.0,1.0
9997,787.086793,785.018482,732.677958,722.580568,223.745255,226.726036,89.784324,87.503559,89.757217,89.826107,...,-10.097390,-2.068311,2.980780,-4.265735,-1.839192,1.0,0.0,0.0,0.0,0.0
9998,850.027606,845.226598,774.510375,793.817333,115.348739,110.382150,87.331351,89.311389,84.625393,85.116316,...,19.306958,-4.801008,-4.966590,-101.104098,-1.986409,1.0,0.0,0.0,0.0,0.0


In [4]:
df_gan

,P_Z0_before,P_Z0_after,P_Z1_before,P_Z1_after,P_WH_before,P_WH_after,T_Z0_before,T_Z0_after,T_Z1_before,T_Z1_after,P_bottom_before,P_bottom_after,T_WH_before,T_WH_after,failure
0,809.623169,810.731628,774.349915,743.234802,254.993607,235.418289,87.142670,83.617828,84.813690,83.927757,804.413025,784.191040,69.659775,68.144562,1.0
1,798.194641,802.370544,757.592285,735.547424,252.063568,236.000000,85.659622,83.670715,85.462723,83.123856,798.249573,778.745850,69.535324,67.042488,1.0
2,818.481506,847.055542,773.057312,769.048035,623.087891,633.995850,106.328621,91.069641,96.541176,90.217949,633.204407,581.719055,58.857521,56.946575,0.0
3,732.812927,744.685852,660.502197,681.042969,406.143585,391.753632,83.088882,84.033012,85.716240,77.493233,692.733215,699.941528,60.912800,62.955395,1.0
4,853.963745,837.576050,803.128845,781.672302,268.001526,229.451889,87.378784,83.756706,80.675461,86.298561,836.298523,797.531006,75.101418,69.453583,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,844.962524,829.377319,800.394714,772.007629,261.395172,230.846939,86.867081,83.894569,82.335457,85.304985,827.716858,793.296570,73.391495,69.020332,1.0
9996,859.043335,845.364746,804.045593,790.442383,301.439362,245.954224,91.225800,83.944077,76.727547,88.188782,844.699036,812.933044,75.237419,72.528130,1.0
9997,811.806824,804.030151,772.481201,743.309937,263.010284,238.615112,85.600319,82.760300,83.991386,83.100784,802.328247,773.420959,70.548927,65.634872,1.0
9998,858.932861,844.563171,804.007080,789.300842,287.551056,236.249146,89.577263,83.869591,77.850677,87.740730,843.718384,808.251892,75.869232,71.410835,1.0


In [5]:
df_test

,P_bottom_before,P_bottom_after,P_bottom_delta,P_Z0_before,P_Z0_after,P_Z0_delta,P_Z1_before,P_Z1_after,P_Z1_delta,P_WH_before,...,action_result,top_icv_status_before,top_icv_status_after,bottom_icv_status_before,bottom_icv_status_after,command_type_0,command_type_1,command_type_2,command_type_3,command_type_4
0,643.589100,643.608020,0.018920,853.061340,853.080260,0.018920,816.967413,816.986333,0.018920,261.971283,...,1,True,True,True,True,0.0,0.0,1.0,0.0,0.0
1,643.609483,643.626475,0.016992,853.081724,853.098715,0.016992,816.987796,817.004788,0.016992,261.971283,...,1,True,True,True,True,0.0,0.0,1.0,0.0,0.0
2,643.626600,643.643800,0.017200,853.098840,853.116040,0.017200,817.004913,817.022113,0.017200,261.020700,...,1,True,True,True,True,0.0,0.0,1.0,0.0,0.0
3,643.644300,643.659800,0.015500,853.116540,853.132040,0.015500,817.022613,817.038113,0.015500,259.846450,...,1,True,True,True,True,0.0,0.0,1.0,0.0,0.0
4,643.659800,643.673700,0.013900,853.132040,853.145940,0.013900,817.038113,817.052013,0.013900,259.343200,...,1,True,True,True,True,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,554.930564,552.103200,-2.827364,764.402804,761.575440,-2.827364,728.308877,725.481513,-2.827364,160.589585,...,1,True,False,True,False,0.0,0.0,1.0,0.0,0.0
249,552.084726,551.859215,-0.225511,761.556967,761.331456,-0.225511,725.463039,725.237528,-0.225511,159.595826,...,1,True,False,True,False,0.0,0.0,1.0,0.0,0.0
250,551.821387,551.051082,-0.770305,761.293627,760.523322,-0.770305,725.199699,724.429394,-0.770305,160.179751,...,0,False,True,False,True,0.0,0.0,0.0,0.0,1.0
251,551.021986,556.434525,5.412539,760.494226,765.906765,5.412539,724.400298,729.812838,5.412539,162.942267,...,0,False,True,False,True,0.0,0.0,0.0,1.0,0.0


In [6]:
df_sim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   P_Z0_before               10000 non-null  float64
 1   P_Z0_after                10000 non-null  float64
 2   P_Z1_before               10000 non-null  float64
 3   P_Z1_after                10000 non-null  float64
 4   P_WH_before               10000 non-null  float64
 5   P_WH_after                10000 non-null  float64
 6   T_Z0_before               10000 non-null  float64
 7   T_Z0_after                10000 non-null  float64
 8   T_Z1_before               10000 non-null  float64
 9   T_Z1_after                10000 non-null  float64
 10  top_icv_status_before     10000 non-null  bool   
 11  top_icv_status_after      10000 non-null  bool   
 12  bottom_icv_status_before  10000 non-null  bool   
 13  bottom_icv_status_after   10000 non-null  bool   
 14  P_botto

In [7]:
df_gan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   P_Z0_before      10000 non-null  float32
 1   P_Z0_after       10000 non-null  float32
 2   P_Z1_before      10000 non-null  float32
 3   P_Z1_after       10000 non-null  float32
 4   P_WH_before      10000 non-null  float32
 5   P_WH_after       10000 non-null  float32
 6   T_Z0_before      10000 non-null  float32
 7   T_Z0_after       10000 non-null  float32
 8   T_Z1_before      10000 non-null  float32
 9   T_Z1_after       10000 non-null  float32
 10  P_bottom_before  10000 non-null  float32
 11  P_bottom_after   10000 non-null  float32
 12  T_WH_before      10000 non-null  float32
 13  T_WH_after       10000 non-null  float32
 14  failure          10000 non-null  float32
dtypes: float32(15)
memory usage: 586.1 KB


In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 235 entries, 0 to 253
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   P_bottom_before           235 non-null    float64
 1   P_bottom_after            235 non-null    float64
 2   P_bottom_delta            235 non-null    float64
 3   P_Z0_before               235 non-null    float64
 4   P_Z0_after                235 non-null    float64
 5   P_Z0_delta                235 non-null    float64
 6   P_Z1_before               235 non-null    float64
 7   P_Z1_after                235 non-null    float64
 8   P_Z1_delta                235 non-null    float64
 9   P_WH_before               235 non-null    float64
 10  P_WH_after                235 non-null    float64
 11  P_WH_delta                235 non-null    float64
 12  T_Z0_before               235 non-null    float64
 13  T_Z0_after                235 non-null    float64
 14  T_Z0_delta

## Data engineering

In [9]:
# Calculating the difference (delta) after - before for each measurable variable for both df_gan and df_sim
float_vars = set([col.split('_')[0]+'_'+col.split('_')[1] for col in list(df_gan.drop(columns='failure').columns)])
for name in float_vars:
    cols = [col for col in list(df_gan.columns) if col.startswith(name)]
    df_gan[name+'_delta'] = df_gan[cols[1]] - df_gan[cols[0]]

In [10]:
df_gan

,P_Z0_before,P_Z0_after,P_Z1_before,P_Z1_after,P_WH_before,P_WH_after,T_Z0_before,T_Z0_after,T_Z1_before,T_Z1_after,...,T_WH_before,T_WH_after,failure,P_Z1_delta,P_WH_delta,T_Z0_delta,T_Z1_delta,P_bottom_delta,P_Z0_delta,T_WH_delta
0,809.623169,810.731628,774.349915,743.234802,254.993607,235.418289,87.142670,83.617828,84.813690,83.927757,...,69.659775,68.144562,1.0,-31.115112,-19.575317,-3.524841,-0.885933,-20.221985,1.108459,-1.515213
1,798.194641,802.370544,757.592285,735.547424,252.063568,236.000000,85.659622,83.670715,85.462723,83.123856,...,69.535324,67.042488,1.0,-22.044861,-16.063568,-1.988907,-2.338867,-19.503723,4.175903,-2.492836
2,818.481506,847.055542,773.057312,769.048035,623.087891,633.995850,106.328621,91.069641,96.541176,90.217949,...,58.857521,56.946575,0.0,-4.009277,10.907959,-15.258980,-6.323227,-51.485352,28.574036,-1.910946
3,732.812927,744.685852,660.502197,681.042969,406.143585,391.753632,83.088882,84.033012,85.716240,77.493233,...,60.912800,62.955395,1.0,20.540771,-14.389954,0.944130,-8.223007,7.208313,11.872925,2.042595
4,853.963745,837.576050,803.128845,781.672302,268.001526,229.451889,87.378784,83.756706,80.675461,86.298561,...,75.101418,69.453583,1.0,-21.456543,-38.549637,-3.622078,5.623100,-38.767517,-16.387695,-5.647835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,844.962524,829.377319,800.394714,772.007629,261.395172,230.846939,86.867081,83.894569,82.335457,85.304985,...,73.391495,69.020332,1.0,-28.387085,-30.548233,-2.972511,2.969528,-34.420288,-15.585205,-4.371162
9996,859.043335,845.364746,804.045593,790.442383,301.439362,245.954224,91.225800,83.944077,76.727547,88.188782,...,75.237419,72.528130,1.0,-13.603210,-55.485138,-7.281723,11.461235,-31.765991,-13.678589,-2.709290
9997,811.806824,804.030151,772.481201,743.309937,263.010284,238.615112,85.600319,82.760300,83.991386,83.100784,...,70.548927,65.634872,1.0,-29.171265,-24.395172,-2.840019,-0.890602,-28.907288,-7.776672,-4.914055
9998,858.932861,844.563171,804.007080,789.300842,287.551056,236.249146,89.577263,83.869591,77.850677,87.740730,...,75.869232,71.410835,1.0,-14.706238,-51.301910,-5.707672,9.890053,-35.466492,-14.369690,-4.458397


In [11]:
df_gan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   P_Z0_before      10000 non-null  float32
 1   P_Z0_after       10000 non-null  float32
 2   P_Z1_before      10000 non-null  float32
 3   P_Z1_after       10000 non-null  float32
 4   P_WH_before      10000 non-null  float32
 5   P_WH_after       10000 non-null  float32
 6   T_Z0_before      10000 non-null  float32
 7   T_Z0_after       10000 non-null  float32
 8   T_Z1_before      10000 non-null  float32
 9   T_Z1_after       10000 non-null  float32
 10  P_bottom_before  10000 non-null  float32
 11  P_bottom_after   10000 non-null  float32
 12  T_WH_before      10000 non-null  float32
 13  T_WH_after       10000 non-null  float32
 14  failure          10000 non-null  float32
 15  P_Z1_delta       10000 non-null  float32
 16  P_WH_delta       10000 non-null  float32
 17  T_Z0_delta   

In [12]:
df_test['action_result'].value_counts()

1    212
0     23
Name: action_result, dtype: int64

In [13]:
## action_result is 1 if the valve action is successful and 0 if it is failed in df_test and df_sim
## Adjusting the label to be 1 if a failure occurs
df_test['failure'] = abs(df_test['action_result'] - 1)
df_sim['failure'] = abs(df_sim['action_result'] - 1)

In [14]:
df_test.drop(columns='action_result', inplace=True)
df_sim.drop(columns='action_result', inplace=True)

In [15]:
df_test['failure'].value_counts()

0    212
1     23
Name: failure, dtype: int64

In [16]:
## Setting train and test sets for both methods
## The test dataset is formed by real data with real failure labels

X_train_sim = df_sim.drop(columns='failure').values.astype('float')
y_train_sim = df_sim['failure'].values.astype('int')

X_train_gan = df_gan.drop(columns='failure').values.astype('float')
y_train_gan = df_gan['failure'].values.astype('int')

X_test_sim = df_test.drop(columns='failure').values.astype('float')
y_test_sim = df_test['failure'].values.astype('int')

X_test_gan = df_test.drop(columns=['top_icv_status_before',
       'top_icv_status_after', 'bottom_icv_status_before',
       'bottom_icv_status_after', 'command_type_0', 'command_type_1',
       'command_type_2', 'command_type_3', 'command_type_4', 'failure']).values.astype('float')
y_test_gan = df_test['failure'].values.astype('int')

In [17]:
X_train_sim.shape

(10000, 30)

In [18]:
X_test_sim.shape

(235, 30)

In [19]:
X_train_gan.shape

(10000, 21)

In [20]:
X_test_gan.shape

(235, 21)

In [21]:
X_test_gan.dtype

dtype('float64')

In [22]:
X_train_gan.dtype

dtype('float64')

In [23]:
X_train_sim.dtype

dtype('float64')

## Setting the Experiment

In [24]:
experiment_name = '2024-01-16_SIMULATED_vs_GAN'
n_experiments = 30

In [25]:
experiment = mlflow.set_experiment(experiment_name)

2024/01/16 14:35:09 INFO mlflow.tracking.fluent: Experiment with name '2024-01-16_SIMULATED_vs_GAN' does not exist. Creating a new experiment.


In [26]:
data_sets = {
    'sim': {
        'X_train': X_train_sim,
        'X_test': X_test_sim,
        'y_train': y_train_sim,
        'y_test': y_test_sim
    },
    'gan': {
        'X_train': X_train_gan,
        'X_test': X_test_gan,
        'y_train': y_train_gan,
        'y_test': y_test_gan
    }
}

In [27]:
models = {
    'DT': DecisionTreeClassifier(),
    'RF': RandomForestClassifier(),
    'KNN': KNeighborsClassifier(),
    'GNB': GaussianNB(),
    'SVC': SVC(),
    'AdaBoost': AdaBoostClassifier(),
    'MLP': MLPClassifier(),
    'LogReg': LogisticRegression(),
    'XGB': XGBRFClassifier(),
    'Dummy': DummyClassifier(strategy='uniform')
}

In [28]:
for i in range(n_experiments):
    
    for data_label, data in data_sets.items():

        for mod_label, model in models.items():
            
            print('-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-')
            print(f'----> Experiment {i+1} of {n_experiments}')
            print('-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-')
            
            with mlflow.start_run(experiment_id=experiment.experiment_id if experiment else None):
            
                # Registering training parameters

                mlflow.log_param('data_set', data_label)
                mlflow.log_param('algorithm', mod_label)
                mlflow.log_param('model', f'{data_label}_{mod_label}')

                print(f'Training model {data_label}_{mod_label}...')
                model.fit(data['X_train'], data['y_train'])

                print(f'Making predictions with model {data_label}_{mod_label}...')
                y_pred = model.predict(data['X_test'])

                print(f'Calculating metrics for model {data_label}_{mod_label}...')
                class_report = classification_report(y_pred, data['y_test'], output_dict=True)

                dic_metrics = pd.json_normalize(class_report, sep='_').to_dict()

                # Registering metrics
                for k, v in dic_metrics.items():
                    mlflow.log_metric(k, v[0])

                print(dic_metrics)
                print('Metrics registered successfully.')
                print('*'*40)

-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-
----> Experiment 1 of 30
-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-
Training model sim_DT...
Making predictions with model sim_DT...
Calculating metrics for model sim_DT...
{'accuracy': {0: 0.9531914893617022}, '0_precision': {0: 0.9481132075471698}, '0_recall': {0: 1.0}, '0_f1-score': {0: 0.973365617433414}, '0_support': {0: 201}, '1_precision': {0: 1.0}, '1_recall': {0: 0.6764705882352942}, '1_f1-score': {0: 0.8070175438596492}, '1_support': {0: 34}, 'macro avg_precision': {0: 0.9740566037735849}, 'macro avg_recall': {0: 0.8382352941176471}, 'macro avg_f1-score': {0: 0.8901915806465316}, 'macro avg_support': {0: 235}, 'weighted avg_precision': {0: 0.9556202328382176}, 'weighted avg_recall': {0: 0.9531914893617022}, 'weighted avg_f1-score': {0: 0.9492982365759332}, 'weighted avg_support': {0: 235}}
Metrics registered successfully.
****************************************
-x-x-x-x-x-x-x-x-x-x-x-x-x

Training model gan_DT...
Making predictions with model gan_DT...
Calculating metrics for model gan_DT...
{'accuracy': {0: 0.902127659574468}, '0_precision': {0: 1.0}, '0_recall': {0: 0.902127659574468}, '0_f1-score': {0: 0.9485458612975392}, '0_support': {0: 235}, '1_precision': {0: 0.0}, '1_recall': {0: 0.0}, '1_f1-score': {0: 0.0}, '1_support': {0: 0}, 'macro avg_precision': {0: 0.5}, 'macro avg_recall': {0: 0.451063829787234}, 'macro avg_f1-score': {0: 0.4742729306487696}, 'macro avg_support': {0: 235}, 'weighted avg_precision': {0: 1.0}, 'weighted avg_recall': {0: 0.902127659574468}, 'weighted avg_f1-score': {0: 0.9485458612975392}, 'weighted avg_support': {0: 235}}
Metrics registered successfully.
****************************************
-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-
----> Experiment 1 of 30
-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-
Training model gan_RF...
Making predictions with model gan_RF...
Calculating metrics for m

Making predictions with model sim_DT...
Calculating metrics for model sim_DT...
{'accuracy': {0: 0.9234042553191489}, '0_precision': {0: 0.9575471698113207}, '0_recall': {0: 0.9575471698113207}, '0_f1-score': {0: 0.9575471698113207}, '0_support': {0: 212}, '1_precision': {0: 0.6086956521739131}, '1_recall': {0: 0.6086956521739131}, '1_f1-score': {0: 0.6086956521739131}, '1_support': {0: 23}, 'macro avg_precision': {0: 0.7831214109926169}, 'macro avg_recall': {0: 0.7831214109926169}, 'macro avg_f1-score': {0: 0.7831214109926169}, 'macro avg_support': {0: 235}, 'weighted avg_precision': {0: 0.9234042553191489}, 'weighted avg_recall': {0: 0.9234042553191489}, 'weighted avg_f1-score': {0: 0.9234042553191489}, 'weighted avg_support': {0: 235}}
Metrics registered successfully.
****************************************
-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-
----> Experiment 2 of 30
-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-
Training model sim_R

Training model gan_DT...
Making predictions with model gan_DT...
Calculating metrics for model gan_DT...
{'accuracy': {0: 0.902127659574468}, '0_precision': {0: 1.0}, '0_recall': {0: 0.902127659574468}, '0_f1-score': {0: 0.9485458612975392}, '0_support': {0: 235}, '1_precision': {0: 0.0}, '1_recall': {0: 0.0}, '1_f1-score': {0: 0.0}, '1_support': {0: 0}, 'macro avg_precision': {0: 0.5}, 'macro avg_recall': {0: 0.451063829787234}, 'macro avg_f1-score': {0: 0.4742729306487696}, 'macro avg_support': {0: 235}, 'weighted avg_precision': {0: 1.0}, 'weighted avg_recall': {0: 0.902127659574468}, 'weighted avg_f1-score': {0: 0.9485458612975392}, 'weighted avg_support': {0: 235}}
Metrics registered successfully.
****************************************
-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-
----> Experiment 2 of 30
-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-
Training model gan_RF...
Making predictions with model gan_RF...
Calculating metrics for m

Making predictions with model sim_DT...
Calculating metrics for model sim_DT...
{'accuracy': {0: 0.9276595744680851}, '0_precision': {0: 0.9858490566037735}, '0_recall': {0: 0.9372197309417041}, '0_f1-score': {0: 0.9609195402298851}, '0_support': {0: 223}, '1_precision': {0: 0.391304347826087}, '1_recall': {0: 0.75}, '1_f1-score': {0: 0.5142857142857143}, '1_support': {0: 12}, 'macro avg_precision': {0: 0.6885767022149303}, 'macro avg_recall': {0: 0.843609865470852}, 'macro avg_f1-score': {0: 0.7376026272577998}, 'macro avg_support': {0: 235}, 'weighted avg_precision': {0: 0.955489326793849}, 'weighted avg_recall': {0: 0.9276595744680851}, 'weighted avg_f1-score': {0: 0.938112706564651}, 'weighted avg_support': {0: 235}}
Metrics registered successfully.
****************************************
-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-
----> Experiment 3 of 30
-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-
Training model sim_RF...
Making predic

...

Making predictions with model sim_DT...
Calculating metrics for model sim_DT...
{'accuracy': {0: 0.9531914893617022}, '0_precision': {0: 0.9481132075471698}, '0_recall': {0: 1.0}, '0_f1-score': {0: 0.973365617433414}, '0_support': {0: 201}, '1_precision': {0: 1.0}, '1_recall': {0: 0.6764705882352942}, '1_f1-score': {0: 0.8070175438596492}, '1_support': {0: 34}, 'macro avg_precision': {0: 0.9740566037735849}, 'macro avg_recall': {0: 0.8382352941176471}, 'macro avg_f1-score': {0: 0.8901915806465316}, 'macro avg_support': {0: 235}, 'weighted avg_precision': {0: 0.9556202328382176}, 'weighted avg_recall': {0: 0.9531914893617022}, 'weighted avg_f1-score': {0: 0.9492982365759332}, 'weighted avg_support': {0: 235}}
Metrics registered successfully.
****************************************
-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-
----> Experiment 29 of 30
-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-
Training model sim_RF...
Making predictions with m

Training model gan_DT...
Making predictions with model gan_DT...
Calculating metrics for model gan_DT...
{'accuracy': {0: 0.902127659574468}, '0_precision': {0: 1.0}, '0_recall': {0: 0.902127659574468}, '0_f1-score': {0: 0.9485458612975392}, '0_support': {0: 235}, '1_precision': {0: 0.0}, '1_recall': {0: 0.0}, '1_f1-score': {0: 0.0}, '1_support': {0: 0}, 'macro avg_precision': {0: 0.5}, 'macro avg_recall': {0: 0.451063829787234}, 'macro avg_f1-score': {0: 0.4742729306487696}, 'macro avg_support': {0: 235}, 'weighted avg_precision': {0: 1.0}, 'weighted avg_recall': {0: 0.902127659574468}, 'weighted avg_f1-score': {0: 0.9485458612975392}, 'weighted avg_support': {0: 235}}
Metrics registered successfully.
****************************************
-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-
----> Experiment 29 of 30
-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-
Training model gan_RF...
Making predictions with model gan_RF...
Calculating metrics for 

Making predictions with model sim_DT...
Calculating metrics for model sim_DT...
{'accuracy': {0: 0.902127659574468}, '0_precision': {0: 1.0}, '0_recall': {0: 0.902127659574468}, '0_f1-score': {0: 0.9485458612975392}, '0_support': {0: 235}, '1_precision': {0: 0.0}, '1_recall': {0: 0.0}, '1_f1-score': {0: 0.0}, '1_support': {0: 0}, 'macro avg_precision': {0: 0.5}, 'macro avg_recall': {0: 0.451063829787234}, 'macro avg_f1-score': {0: 0.4742729306487696}, 'macro avg_support': {0: 235}, 'weighted avg_precision': {0: 1.0}, 'weighted avg_recall': {0: 0.902127659574468}, 'weighted avg_f1-score': {0: 0.9485458612975392}, 'weighted avg_support': {0: 235}}
Metrics registered successfully.
****************************************
-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-
----> Experiment 30 of 30
-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-
Training model sim_RF...
Making predictions with model sim_RF...
Calculating metrics for model sim_RF...
{'accurac

{'accuracy': {0: 0.5276595744680851}, '0_precision': {0: 0.5283018867924528}, '0_recall': {0: 0.9105691056910569}, '0_f1-score': {0: 0.6686567164179104}, '0_support': {0: 123}, '1_precision': {0: 0.5217391304347826}, '1_recall': {0: 0.10714285714285714}, '1_f1-score': {0: 0.17777777777777778}, '1_support': {0: 112}, 'macro avg_precision': {0: 0.5250205086136177}, 'macro avg_recall': {0: 0.508855981416957}, 'macro avg_f1-score': {0: 0.4232172470978441}, 'macro avg_support': {0: 235}, 'weighted avg_precision': {0: 0.52517410503901}, 'weighted avg_recall': {0: 0.5276595744680851}, 'weighted avg_f1-score': {0: 0.43470590310857055}, 'weighted avg_support': {0: 235}}
Metrics registered successfully.
****************************************
-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-
----> Experiment 30 of 30
-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-
Training model gan_DT...
Making predictions with model gan_DT...
Calculating metrics for model gan

Making predictions with model gan_XGB...
Calculating metrics for model gan_XGB...
{'accuracy': {0: 0.902127659574468}, '0_precision': {0: 1.0}, '0_recall': {0: 0.902127659574468}, '0_f1-score': {0: 0.9485458612975392}, '0_support': {0: 235}, '1_precision': {0: 0.0}, '1_recall': {0: 0.0}, '1_f1-score': {0: 0.0}, '1_support': {0: 0}, 'macro avg_precision': {0: 0.5}, 'macro avg_recall': {0: 0.451063829787234}, 'macro avg_f1-score': {0: 0.4742729306487696}, 'macro avg_support': {0: 235}, 'weighted avg_precision': {0: 1.0}, 'weighted avg_recall': {0: 0.902127659574468}, 'weighted avg_f1-score': {0: 0.9485458612975392}, 'weighted avg_support': {0: 235}}
Metrics registered successfully.
****************************************
-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-
----> Experiment 30 of 30
-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-
Training model gan_Dummy...
Making predictions with model gan_Dummy...
Calculating metrics for model gan_Dummy..